In [6]:
import glob
import pandas as pd

# get list of all protein IDs
protein_list = []
for file in glob.glob('Data/*.csv'):
    protein_list.append(file[5:9])

# generate a list of all dataframes
dfs_list = []

for protein in protein_list:
  df = pd.read_csv("Data/" + protein + "_seq_lab.csv")
  df["protein_id"] = protein
  dfs_list.append(df)

# merge all dataframes to one dataframe
data_all = pd.concat(dfs_list)

In [7]:
AAs = "CAGPSTNDEQRHKILMVFWY"
groups = [AA for AA in AAs]

In [8]:
import re
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import torch
import matplotlib.pyplot as plt

def count_mer(mer, seq):
    '''
    Counts the number of times a substring mer
    ocurrs in the sequence seq (including overlapping
    occurrences)

    sample use: count_mer("GGG", "AGGGCGGG") => 2
    '''

    k = len(mer)
    count = 0
    for i in range(0, len(seq)-k+1):
        if mer == seq[i:i+k]:
            count = count + 1
    return count
    
def kmer_count(mers, seq):
    '''
    Return a list of the number of times each possible k-mer appears
    in seq, including overlapping occurrences.
    '''

    rv = []
    for m in mers:
        cnt = count_mer(m, seq)        
        rv.append(cnt)
    return rv

def get_counts(mers, seqs):
    counts = [kmer_count(mers, seq) for seq in seqs]
    return np.array(counts)

def recode_seq(text, args_dict):
    for key in args_dict.keys():
        text = text.replace(key, str(args_dict[key]))
    return text

class Preprocessing:

    def __init__(self, data, args_dict, num_words, seq_len=1024):
        self.data = data
        self.args_dict = args_dict
        self.seq_len = seq_len
        self.num_words = num_words        
        self.vocabulary = None
        self.x_tokenized = None
        self.x_padded = None
        self.x_raw = None
        

        self.x_train = None
        self.x_test = None
        self.y_train = None
        self.y_test = None
        
        self.X_counts_ = {}

    def load_data(self):
        # Reads the raw csv file and split into
        # sentences (x) and target (y)

        df = self.data

        self.x_raw = df['seqs'].values
        self.y = torch.tensor(df['labels'].values).cuda()        

    def text_tokenization(self):
        # Tokenizes each sentence by implementing the nltk tool
        self.x_converted = [recode_seq(x, self.args_dict) for x in self.x_raw]

    def build_vocabulary(self):
        # Builds the vocabulary and keeps the "x" most frequent words
        self.vocabulary = dict()
        fdist = nltk.FreqDist()

        for sentence in self.x_converted:
            for word in sentence:
                fdist[word] += 1

        common_words = fdist.most_common(self.num_words)

        for idx, word in enumerate(common_words):
            self.vocabulary[word[0]] = (idx+1)
  
    def word_to_idx(self):
        # By using the dictionary (vocabulary), it is transformed
        # each token into its index based representation

        self.x_tokenized = list()

        for sentence in self.x_converted:
            temp_sentence = list()
            for word in sentence:
                if word in self.vocabulary.keys():
                    temp_sentence.append(self.vocabulary[word])
            self.x_tokenized.append(temp_sentence)
	      
    def padding_sentences(self):
        # Each sentence which does not fulfill the required len
        # it's padded with the index 0

        pad_idx = 0
        self.x_padded = []
        
        for sentence in self.x_tokenized:
            if len(sentence) > self.seq_len:
                sentence = sentence[:self.seq_len]
            else: pass
            while len(sentence) < self.seq_len:
                sentence.insert(len(sentence), pad_idx)
            self.x_padded.append(sentence)
        self.X = torch.tensor(np.array(self.x_padded)).cuda()
        
    def mer_counts(self, k):
        alphabet_reduced = list(range(len(groups)))
        alphabet_reduced = [str(a) for a in alphabet_reduced]

        k_mers = list(product(*[alphabet_reduced for i in range(k)]))
        k_mers = [("").join(list(k_mer)) for k_mer in k_mers]
        
        mer_counts = get_counts(k_mers, self.x_converted)
        self.X_counts_[k] = torch.tensor(mer_counts).cuda()
        
        
    def prep_data(self, train_prots, counts):

        train_bool = [ID in set(train_prots) for ID in self.data.protein_id]
        test_bool = [ID not in set(train_prots) for ID in self.data.protein_id]
        
        if counts:
            X = self.X_counts
            # X = X/X.sum(1).unsqueeze(1)
        else: X = self.X
        train_x, train_y = X[train_bool], self.y[train_bool]
        
        test_x, test_y = X[test_bool], self.y[test_bool]    

        data = {}
        data["x_train"] = train_x
        data["y_train"] = train_y.float()
        data["x_test"] = test_x
        # data["y_test"] = test_y[np.random.permutation(len(test_y))]
        data["y_test"] = test_y.float()

        return data


In [9]:
from itertools import product

k = 2

alphabet_reduced = list(range(len(groups)))
alphabet_reduced = [str(a) for a in alphabet_reduced]

k_mers = list(product(*[alphabet_reduced for i in range(k)]))
k_mers = [("").join(list(k_mer)) for k_mer in k_mers]

AA2G = {}
for i, group in enumerate(groups):
    for letter in group:
        AA2G[letter] = i

In [10]:
pre = Preprocessing(data_all, AA2G, num_words=25, seq_len=1024)
pre.load_data()
pre.text_tokenization()
pre.build_vocabulary()
pre.word_to_idx()
pre.padding_sentences()

In [11]:
pre.mer_counts(1)
pre.mer_counts(2)

### Logistic regression

In [12]:
results = {}

#### Dimers

In [13]:
pre.X_counts = pre.X_counts_[2]

from sklearn.linear_model import LogisticRegression


auroc_list = []
coef_list = []
complexity_param = 10


for i in range(len(protein_list)):
    train_prots = pd.Series(protein_list).drop(i).tolist()
    data = pre.prep_data(train_prots, True)
    model = LogisticRegression(random_state=0, penalty='l1', solver="liblinear", C=complexity_param)
    clf = model.fit(data['x_train'].cpu().numpy(), data['y_train'].cpu().numpy())
    predict_y = clf.predict(data['x_test'].cpu().numpy())
    predict_p = clf.predict_proba(data['x_test'].cpu().numpy())    
    auroc = roc_auc_score(data['y_test'].cpu().numpy(), predict_y)
    auroc_list.append(auroc)
    coef_list.append(clf.coef_)

results["Dimer"] = auroc_list

#### Monomers

In [14]:
pre.X_counts = pre.X_counts_[1]

auroc_list = []
complexity_param = 1
for i in range(len(protein_list)):
    train_prots = pd.Series(protein_list).drop(i).tolist()
    data = pre.prep_data(train_prots, True)
    model = LogisticRegression(random_state=0, penalty='l2', solver="liblinear", C=complexity_param)
    clf = model.fit(data['x_train'].cpu().numpy(), data['y_train'].cpu().numpy())
    predict_y = clf.predict(data['x_test'].cpu().numpy())
    predict_p = clf.predict_proba(data['x_test'].cpu().numpy())    
    auroc = roc_auc_score(data['y_test'].cpu().numpy(), predict_y)
    auroc_list.append(auroc)

results["Monomer"] = auroc_list


In [15]:
results["CNN"] = pd.read_csv("AUROC_list_CNN.csv").to_numpy().flatten()
x_labels = ["Monomer", "Dimer", "CNN"]

plotdata = [results["Monomer"], results["Dimer"], results["CNN"]]
widths = [0.5, 0.5, 0.5]  


# Create the boxplot with narrower boxes
fig, ax = plt.subplots(figsize=(8, 8))

# Adjust the width of the boxes (e.g., set width to 0.5 for narrower boxes)
boxplot = ax.boxplot(plotdata, patch_artist=True, widths=0.2)

# Set the x-labels and y-axis label as needed
ax.set_xticklabels(x_labels, fontsize=20)
ax.set_ylabel('Test AUROC', fontsize=20)

# Customize the box colors and edge colors (optional)
for box in boxplot['boxes']:
    box.set(facecolor='gray', edgecolor='black')

median_color = 'orange'
for median in boxplot['medians']:
    median.set(color=median_color, linewidth=3.5)  # Adjust linewidth as needed
    
ax.tick_params(axis='y', labelsize=20)
ax.grid(True, linestyle='--', alpha=0.8)  # You can customize linestyle and alpha as needed

# Save and show the plot
plt.savefig('figs/auroc_box.pdf', dpi=500)
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'AUROC_list_CNN.csv'

### Visualization of counts data

#### Dimer

In [ ]:
data_all[data_all.protein_id == "5296"].to_csv("seq_data.csv")

counts_raw = pre.X_counts_[2]

freqs = counts_raw/counts_raw.sum(1).unsqueeze(1)

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
X = freqs.detach().cpu().numpy()
X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(X)
X_embedded.shape


plotdf = pd.DataFrame({"x1":X_embedded[:,0], "x2": X_embedded[:, 1], "lab": data_all.labels})

plotdf = plotdf.rename(columns={"x1":"tSNE1", "x2": "tSNE2", "lab":"Species"})

plotdf.Species = plotdf.Species.replace({0: "Fungal", 1: "Plant"})

In [16]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches

sns.set(font_scale=1.2)  
sns.set_style("ticks") 

plt.figure(figsize=(8, 8)) 

# Your data plotting code here (e.g., sns.relplot)

sns.relplot(data=plotdf, x='tSNE1', y='tSNE2', hue='Species', aspect=1, alpha=0.5)
# Get the current axes
ax = plt.gca()

# # Add ticks to the x-axis and y-axis
plt.xticks(range(-100, 120, 50))  # Specify the tick positions for the x-axis
plt.yticks(range(-100, 120, 50))  # Specify the tick positions for the x-axis


plt.savefig('figs/dimer_tSNE.pdf', dpi=500)
plt.show()

NameError: name 'plotdf' is not defined

<Figure size 576x576 with 0 Axes>